In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from TrajectoryNet.dataset import EBData
from TrajectoryNet.optimal_transport.emd import earth_mover_distance, interpolate_with_ot

from gp_sinkhorn.SDE_solver import solve_sde_RK
from gp_sinkhorn.MLE_drift import *
from gp_sinkhorn.utils import plot_trajectories_2
import ot

import torch

from celluloid import Camera
from IPython.display import HTML
import math

In [2]:
ds = EBData('pcs', max_dim=5)

Embedding shape  (16819, 100)
Labels shape  (16819,)
Embedding shape  (16819, 100)


In [3]:
frame_0_start, frame_0_end = np.where(ds.labels == 0)[0][0], np.where(ds.labels == 0)[0][-1]
frame_4_start, frame_4_end = np.where(ds.labels == 4)[0][0], np.where(ds.labels == 4)[0][-1]


frame_2_start, frame_2_end = np.where(ds.labels == 2)[0][0], np.where(ds.labels == 2)[0][-1]

frame_1_start, frame_1_end = np.where(ds.labels == 1)[0][0], np.where(ds.labels == 1)[0][-1]


frame_3_start, frame_3_end = np.where(ds.labels == 3)[0][0], np.where(ds.labels == 3)[0][-1]

X_mid_f = ds.get_data()[frame_3_start:frame_3_end+1]
X_mid_list = [ds.get_data()[frame_1_start:frame_1_end+1],ds.get_data()[frame_2_start:frame_2_end+1],ds.get_data()[frame_3_start:frame_3_end+1]]

In [4]:
X_0_f = ds.get_data()[frame_0_start:frame_0_end+1]
X_1_f = ds.get_data()[frame_4_start:]


In [ ]:
#many_results = pd.read_pickle('../data/result_final.pkl')
many_results_gp = pd.read_pickle("../assets/result_dump/40/result_12.pkl") # With gp prior
many_results_no_gp = pd.read_pickle("../assets/result_dump/3/result_final.pkl")
results_plot = np.zeros((len(many_results_gp),2))
for i in range(5):
    print("Iteration number ",i)
    for idx,many_results in enumerate([many_results_gp,many_results_no_gp]):
        result_final_epoch = many_results[i]

        time_forward, zs_forward, time_backward, zs_backward = result_final_epoch

        zs_forward_ = zs_forward[:,:,:-1]
        zs_backward_ = zs_backward[:,:,:-1]
        tmp = []
        for idx_2,place in enumerate([0.25,0.5,0.75]):
            X_mid_f = X_mid_list[idx_2]
            tpi_f = int(math.floor(zs_forward_.shape[1] * place))
            tpi_b = int(math.floor(zs_backward_.shape[1] *(1-place) ))

            emd_f = earth_mover_distance(zs_forward_[:,tpi_f,:], X_mid_f)
            emd_b = earth_mover_distance(zs_backward_[:,tpi_b,:], X_mid_f)
            print(emd_f,emd_b)
            tmp.append(emd_f)
            tmp.append(emd_b)
        results_plot[i,idx] = np.array(tmp).mean()
        print("--")
            
    print("----")

In [ ]:
plt.plot(results_plot[:5,0],label="GP")
plt.plot(results_plot[:5,1])
plt.legend()

In [ ]:
from scipy.spatial.distance import cdist

result_ot = np.zeros((2,3,2))
for idx,(X_0_tmp,X_1_tmp) in enumerate([(X_0_f,X_1_f),(X_1_f,X_0_f)]):

    M = M = ot.dist(X_0_tmp, X_1_tmp)
    M /= M.max()
    n, m = X_0_tmp.shape[0], X_1_tmp.shape[0]

    a, b = np.ones(n,)/n, np.ones(m,)/m
    Wd = ot.emd(a,b ,M, numItermax=1e7) # exact linear program
    #Wd_reg = ot.sinkhorn(a,b ,M, 1.0) 


    for idx_2,d in enumerate([0.25,0.5,0.75]):
        X_mid_f = X_mid_list[idx_2]
        pint = interpolate_with_ot(X_0_tmp, X_1_tmp, Wd, d, size=X_0_tmp.shape[0])
        result_ot[idx,idx_2,0] = earth_mover_distance(pint, X_mid_f)
        Wdrnd = np.random.choice([0, 1],Wd.shape)
        pint_rand = interpolate_with_ot(X_0_tmp, X_1_tmp, Wdrnd, d, size=X_0_tmp.shape[0])
        result_ot[idx,idx_2,1] = earth_mover_distance(pint_rand, X_mid_f)


In [ ]:
mean = result_ot.mean(axis=(0,1))


In [ ]:
plt.plot([mean[0]]*results_plot.shape[0],label='OT')
plt.plot([mean[1]]*results_plot.shape[0],label='OT rand')
plt.plot(results_plot[:5,0],label="GP")
plt.plot(results_plot[:5,1])
plt.legend()